Our objective is to come up with final production/consumption traces that we will use in our simulations.

In [2]:
import pandas as pd
import numpy as np

Let us first import all the data at hand.

In [3]:
df1 = pd.read_csv('04-traces-a.csv', na_values=['\\N', 'NaN'], dtype={'dataid': np.int64, 'gen': np.float64, 'grid': np.float64, 'use': np.float64})
df2 = pd.read_csv('04-traces-b.csv', na_values=['\\N', 'NaN'], dtype={'dataid': np.int64, 'gen': np.float64, 'grid': np.float64, 'use': np.float64})
df3 = pd.read_csv('04-traces-c.csv', na_values=['\\N', 'NaN'], dtype={'dataid': np.int64, 'gen': np.float64, 'grid': np.float64, 'use': np.float64})

Merge all dataframes into one, see: https://stackoverflow.com/a/21435402/2363529

In [4]:
df = pd.concat([df1, df2, df3])

How many rows?

In [5]:
df.shape[0]

7484542

In [6]:
df.dtypes

dataid           int64
local_15min     object
gen            float64
grid           float64
use            float64
dtype: object

In [7]:
df['local_15min'] = pd.to_datetime(df['local_15min'])
df.dtypes

dataid                  int64
local_15min    datetime64[ns]
gen                   float64
grid                  float64
use                   float64
dtype: object

In [8]:
df.head()

,dataid,local_15min,gen,grid,use
0,171,2012-10-24 10:00:00,0.940400,-0.538867,0.401600
1,171,2012-10-24 10:15:00,0.823000,-0.446400,0.376667
2,171,2012-10-24 10:30:00,0.704200,-0.335400,0.368800
3,171,2012-10-24 10:45:00,1.015467,-0.646200,0.369067
4,171,2012-10-24 11:00:00,1.388600,-1.118867,0.269867


Set the proper index, see: https://stackoverflow.com/a/35332222/2363529

In [9]:
df.set_index(['dataid', 'local_15min'], inplace=True)

In [10]:
df.head()

gen      grid       use
dataid local_15min                                      
171    2012-10-24 10:00:00  0.940400 -0.538867  0.401600
       2012-10-24 10:15:00  0.823000 -0.446400  0.376667
       2012-10-24 10:30:00  0.704200 -0.335400  0.368800
       2012-10-24 10:45:00  1.015467 -0.646200  0.369067
       2012-10-24 11:00:00  1.388600 -1.118867  0.269867

We need to figure out what to do with the `NaN` values.

In [11]:
df.loc[187, '2014-06-12 00:45:00']

gen    NaN
grid   NaN
use    NaN
Name: (187, 2014-06-12 00:45:00), dtype: float64

In [12]:
df.loc[171].shape[0]

72028

Let us first identify how many rows we have per `dataid`.

In [13]:
df.index.values[0]

(171, Timestamp('2012-10-24 10:00:00'))

Time for a set comprehension: https://github.com/jakevdp/WhirlwindTourOfPython/blob/6f1daf714fe52a8dde6a288674ba46a7feed8816/11-List-Comprehensions.ipynb

In [14]:
dataids = {i[0] for i in list(df.index.values)}
list(dataids)[:5]

[9729, 5129, 9737, 2072, 545]

In [15]:
dataid_row_count = []
for id in dataids:
    dataid_row_count.append(df.loc[id].shape[0])
dataid_row_count[:5]

[72028, 72028, 72028, 72028, 72028]

What is the frequency of each row count?

See: https://stackoverflow.com/a/45268376/2363529

In [16]:
row_counts, freq_row_counts = np.unique(dataid_row_count, return_counts=True)

In [17]:
row_counts[-1]

72028

In [18]:
freq_row_counts[-1]

74

So we have 74 data IDs with 72028 rows.

That seems good enough, but we need to check for empty rows within those data IDs first.

Which are the data IDs with 72028 rows?

In [19]:
dataids_first_pass = sorted([id for id in dataids if (df.loc[id].shape[0] == row_counts[-1])])
dataids_first_pass[:5]

[171, 370, 545, 585, 744]

In [20]:
len(dataids_first_pass) == freq_row_counts[-1]

True

How many empty elements per data ID trace?

In [21]:
np.count_nonzero(df.loc[dataids_first_pass[0]].isnull().values)

0

In [22]:
empty_elems = {id:np.count_nonzero(df.loc[id].isnull().values) for id in dataids_first_pass}
empty_elems

{171: 0,
 370: 0,
 545: 0,
 585: 0,
 744: 0,
 861: 0,
 890: 0,
 1103: 0,
 1185: 10713,
 1283: 0,
 1642: 15,
 1718: 0,
 1792: 0,
 2072: 0,
 2233: 0,
 2335: 72732,
 2337: 0,
 2470: 0,
 2755: 0,
 2818: 0,
 2925: 0,
 2945: 0,
 2980: 0,
 2986: 0,
 3224: 0,
 3456: 0,
 3527: 0,
 3544: 0,
 3635: 0,
 3719: 0,
 3723: 0,
 3918: 0,
 3935: 0,
 4193: 0,
 4302: 0,
 4357: 0,
 4447: 0,
 4526: 0,
 4767: 45870,
 5129: 0,
 5246: 0,
 5275: 0,
 5357: 0,
 5403: 216084,
 5439: 0,
 5615: 0,
 5738: 0,
 5785: 0,
 5796: 0,
 5817: 0,
 5892: 0,
 5972: 0,
 6266: 0,
 6423: 0,
 6643: 0,
 6990: 0,
 7108: 0,
 7731: 0,
 7739: 0,
 7767: 0,
 7800: 159408,
 7863: 0,
 7989: 0,
 8084: 0,
 8155: 0,
 8626: 0,
 8669: 288,
 8829: 0,
 8890: 130540,
 9019: 126,
 9121: 0,
 9631: 0,
 9729: 156900,
 9737: 138023}

Seems that we have enough data IDs without nil values in order to proceed.

In [23]:
chosen_dataids = [id for id, count in empty_elems.items() if count == 0]
chosen_dataids[:10]

[171, 370, 545, 585, 744, 861, 890, 1103, 1283, 1718]

How many?

In [24]:
len(chosen_dataids)

63

What is the start date?

In [25]:
df.loc[chosen_dataids[0]].iloc[0]

gen     0.940400
grid   -0.538867
use     0.401600
Name: 2012-10-24 10:00:00, dtype: float64

What is the end date?

In [26]:
df.loc[chosen_dataids[0]].iloc[-1]

gen    -0.0050
grid    0.5634
use     0.5634
Name: 2014-11-13 18:45:00, dtype: float64

Let us first create a dataframe that only contains the `chosen_data_ids`.

In [27]:
type(df.loc[chosen_dataids[0]])

pandas.core.frame.DataFrame

In [28]:
df.loc[chosen_dataids[0]].shape

(72028, 3)

In [29]:
df.loc[chosen_dataids[0]].dtypes

gen     float64
grid    float64
use     float64
dtype: object

In [30]:
df.shape

(7484542, 3)

In [31]:
df.head()

gen      grid       use
dataid local_15min                                      
171    2012-10-24 10:00:00  0.940400 -0.538867  0.401600
       2012-10-24 10:15:00  0.823000 -0.446400  0.376667
       2012-10-24 10:30:00  0.704200 -0.335400  0.368800
       2012-10-24 10:45:00  1.015467 -0.646200  0.369067
       2012-10-24 11:00:00  1.388600 -1.118867  0.269867

In [32]:
df.loc[1185].head()

,gen,grid,use
local_15min,,,
2012-10-24 10:00:00,1.368733,-0.975733,0.393000
2012-10-24 10:15:00,2.002400,-1.699267,0.303267
2012-10-24 10:30:00,2.409667,-2.090067,0.319533
2012-10-24 10:45:00,2.168467,-1.846667,0.321733
2012-10-24 11:00:00,1.360133,-0.949867,0.410400


We got:
* the `dataids` set containing all data IDs in `df`, and
* `chosen_data_ids`

We wish to create `deleted_dataids`, the supplement of `chosen_data_ids`.

In [33]:
deleted_dataids = [id for id in dataids if id not in chosen_dataids]

In [34]:
len(deleted_dataids) == len(dataids) - len(chosen_dataids)

True

In [35]:
df.drop(deleted_dataids, inplace=True)

C:\Users\kchrist\AppData\Local\Continuum\miniconda3\envs\test\lib\site-packages\pandas\core\generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [36]:
df.shape

(4537764, 3)

In [37]:
remaining_dataids = {i[0] for i in list(df.index.values)}

In [38]:
len(remaining_dataids)

63

We now have a dataframe with just the data IDs we are interested in.

Let's us add the lower/upper limit market columns.

In [39]:
df.head()

gen      grid       use
dataid local_15min                                      
171    2012-10-24 10:00:00  0.940400 -0.538867  0.401600
       2012-10-24 10:15:00  0.823000 -0.446400  0.376667
       2012-10-24 10:30:00  0.704200 -0.335400  0.368800
       2012-10-24 10:45:00  1.015467 -0.646200  0.369067
       2012-10-24 11:00:00  1.388600 -1.118867  0.269867

In [40]:
df.loc[171, '2012-10-24 10:15:00'].name[1].year

2012

Average annual real-time market prices for the South zone in ERCOT, see: https://www.potomaceconomics.com/wp-content/uploads/2017/06/2016-ERCOT-State-of-the-Market-Report.pdf

In [41]:
lower_limit = {2012:2.8, 2013:3.4, 2014:4.2}

In [42]:
lower_limit[2012]

2.8

In [43]:
lower_limit[df.loc[171, '2012-10-24 10:15:00'].name[1].year]

2.8

In [44]:
def yearly_lower_limit(year):
    return lower_limit[year]

In [45]:
yearly_lower_limit(df.loc[171, '2012-10-24 10:15:00'].name[1].year)

2.8

In [46]:
df['low_lim'] = df.apply(lambda x:yearly_lower_limit(x.name[1].year), axis=1)

In [47]:
df.head()

gen      grid       use  low_lim
dataid local_15min                                               
171    2012-10-24 10:00:00  0.940400 -0.538867  0.401600      2.8
       2012-10-24 10:15:00  0.823000 -0.446400  0.376667      2.8
       2012-10-24 10:30:00  0.704200 -0.335400  0.368800      2.8
       2012-10-24 10:45:00  1.015467 -0.646200  0.369067      2.8
       2012-10-24 11:00:00  1.388600 -1.118867  0.269867      2.8

In [48]:
df.tail()

gen      grid       use  low_lim
dataid local_15min                                               
9631   2014-11-13 17:45:00 -0.025467  0.538733  0.538733      4.2
       2014-11-13 18:00:00 -0.007000  0.538133  0.538133      4.2
       2014-11-13 18:15:00 -0.007000  0.497667  0.497667      4.2
       2014-11-13 18:30:00 -0.007000  0.721733  0.721733      4.2
       2014-11-13 18:45:00 -0.007000  0.784867  0.784867      4.2

Import the upper limit CSV.

In [49]:
upper_limit_df = pd.read_csv('04-market-lim-up.csv', header=None, names= ['date', 'price (c/kWh)'])

In [50]:
upper_limit_df.head()

,date,price (c/kWh)
0,10-Oct,11.62
1,10-Nov,11.32
2,10-Dec,11.25
3,11-Jan,10.70
4,11-Feb,10.65


In [51]:
upper_limit_df.dtypes

date              object
price (c/kWh)    float64
dtype: object

In [52]:
upper_limit_df['date'] = pd.to_datetime(upper_limit_df['date'], format='%y-%b')

In [53]:
upper_limit_df.head()

,date,price (c/kWh)
0,2010-10-01,11.62
1,2010-11-01,11.32
2,2010-12-01,11.25
3,2011-01-01,10.70
4,2011-02-01,10.65


In [54]:
upper_limit_df.dtypes

date             datetime64[ns]
price (c/kWh)           float64
dtype: object

In [55]:
upper_limit_df.set_index('date', inplace=True)

In [56]:
upper_limit_df.head()

,price (c/kWh)
date,
2010-10-01,11.62
2010-11-01,11.32
2010-12-01,11.25
2011-01-01,10.70
2011-02-01,10.65


Create `year` and `month` columns, see: https://stackoverflow.com/a/13652262/2363529

In [57]:
upper_limit_df.index.year

Int64Index([2010, 2010, 2010, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011,
            2011, 2011, 2011, 2011, 2012, 2012, 2012, 2012, 2012, 2012, 2012,
            2012, 2012, 2012, 2012, 2012, 2013, 2013, 2013, 2013, 2013, 2013,
            2013, 2013, 2013, 2013, 2013, 2013, 2014, 2014, 2014, 2014, 2014,
            2014, 2014, 2014, 2014, 2014, 2014],
           dtype='int64', name='date')

In [58]:
upper_limit_df = upper_limit_df.assign(year=upper_limit_df.index.year, month=upper_limit_df.index.month)

In [59]:
upper_limit_df.head()

,price (c/kWh),year,month
date,,,
2010-10-01,11.62,2010,10
2010-11-01,11.32,2010,11
2010-12-01,11.25,2010,12
2011-01-01,10.70,2011,1
2011-02-01,10.65,2011,2


In [60]:
upper_limit_df.set_index(['year', 'month'], inplace=True)

In [61]:
upper_limit_df.head()

price (c/kWh)
year month               
2010 10             11.62
     11             11.32
     12             11.25
2011 1              10.70
     2              10.65

In [62]:
upper_limit_df.loc[2010, 10].data[0]

11.62

In [63]:
df.head()

gen      grid       use  low_lim
dataid local_15min                                               
171    2012-10-24 10:00:00  0.940400 -0.538867  0.401600      2.8
       2012-10-24 10:15:00  0.823000 -0.446400  0.376667      2.8
       2012-10-24 10:30:00  0.704200 -0.335400  0.368800      2.8
       2012-10-24 10:45:00  1.015467 -0.646200  0.369067      2.8
       2012-10-24 11:00:00  1.388600 -1.118867  0.269867      2.8

In [64]:
df['upper_lim'] = df.apply(lambda x:upper_limit_df.loc[x.name[1].year,x.name[1].month].data[0], axis=1)

In [65]:
df.head()

gen      grid       use  low_lim  upper_lim
dataid local_15min                                                          
171    2012-10-24 10:00:00  0.940400 -0.538867  0.401600      2.8      11.16
       2012-10-24 10:15:00  0.823000 -0.446400  0.376667      2.8      11.16
       2012-10-24 10:30:00  0.704200 -0.335400  0.368800      2.8      11.16
       2012-10-24 10:45:00  1.015467 -0.646200  0.369067      2.8      11.16
       2012-10-24 11:00:00  1.388600 -1.118867  0.269867      2.8      11.16

In [66]:
upper_limit_df.loc[2012, 10]

price (c/kWh)    11.16
Name: (2012, 10), dtype: float64

In [67]:
df.tail()

gen      grid       use  low_lim  upper_lim
dataid local_15min                                                          
9631   2014-11-13 17:45:00 -0.025467  0.538733  0.538733      4.2      11.98
       2014-11-13 18:00:00 -0.007000  0.538133  0.538133      4.2      11.98
       2014-11-13 18:15:00 -0.007000  0.497667  0.497667      4.2      11.98
       2014-11-13 18:30:00 -0.007000  0.721733  0.721733      4.2      11.98
       2014-11-13 18:45:00 -0.007000  0.784867  0.784867      4.2      11.98

In [68]:
upper_limit_df.loc[2014, 11]

price (c/kWh)    11.98
Name: (2014, 11), dtype: float64

Looks like we're done.

Let's save three versions of this dataframe.

In [69]:
df.to_csv('04-final-trace-full.csv')

In [70]:
df_lite = df.reset_index(level='local_15min', drop=True)

In [71]:
df_lite.head()

,gen,grid,use,low_lim,upper_lim
dataid,,,,,
171,0.940400,-0.538867,0.401600,2.8,11.16
171,0.823000,-0.446400,0.376667,2.8,11.16
171,0.704200,-0.335400,0.368800,2.8,11.16
171,1.015467,-0.646200,0.369067,2.8,11.16
171,1.388600,-1.118867,0.269867,2.8,11.16


In [73]:
df_lite.tail()

,gen,grid,use,low_lim,upper_lim
dataid,,,,,
9631,-0.025467,0.538733,0.538733,4.2,11.98
9631,-0.007000,0.538133,0.538133,4.2,11.98
9631,-0.007000,0.497667,0.497667,4.2,11.98
9631,-0.007000,0.721733,0.721733,4.2,11.98
9631,-0.007000,0.784867,0.784867,4.2,11.98


In [74]:
df_lite.to_csv('04-final-trace-lite.csv')

In [79]:
df_year = df.reset_index(level='local_15min')

In [80]:
df_year.head()

,local_15min,gen,grid,use,low_lim,upper_lim
dataid,,,,,,
171,2012-10-24 10:00:00,0.940400,-0.538867,0.401600,2.8,11.16
171,2012-10-24 10:15:00,0.823000,-0.446400,0.376667,2.8,11.16
171,2012-10-24 10:30:00,0.704200,-0.335400,0.368800,2.8,11.16
171,2012-10-24 10:45:00,1.015467,-0.646200,0.369067,2.8,11.16
171,2012-10-24 11:00:00,1.388600,-1.118867,0.269867,2.8,11.16


In [81]:
df_year.dtypes

local_15min    datetime64[ns]
gen                   float64
grid                  float64
use                   float64
low_lim               float64
upper_lim             float64
dtype: object

In [85]:
df_year = df_year[df_year['local_15min'].dt.year == int(2013)]

In [86]:
df_year.head()

,local_15min,gen,grid,use,low_lim,upper_lim
dataid,,,,,,
171,2013-01-01 00:00:00,-0.005,0.878267,0.878267,3.4,10.84
171,2013-01-01 00:15:00,-0.005,0.607600,0.607600,3.4,10.84
171,2013-01-01 00:30:00,-0.005,0.717600,0.717600,3.4,10.84
171,2013-01-01 00:45:00,-0.005,0.642333,0.642333,3.4,10.84
171,2013-01-01 01:00:00,-0.005,0.683000,0.683000,3.4,10.84


In [87]:
df_year.tail()

,local_15min,gen,grid,use,low_lim,upper_lim
dataid,,,,,,
9631,2013-12-31 22:45:00,-0.0060,0.814000,0.814000,3.4,11.27
9631,2013-12-31 23:00:00,-0.0060,0.641333,0.641333,3.4,11.27
9631,2013-12-31 23:15:00,-0.0060,0.473400,0.473400,3.4,11.27
9631,2013-12-31 23:30:00,-0.0060,0.682800,0.682800,3.4,11.27
9631,2013-12-31 23:45:00,-0.0066,1.283200,1.283200,3.4,11.27


In [88]:
df_year.drop(columns=['local_15min'], inplace=True)

In [89]:
df_year.head()

,gen,grid,use,low_lim,upper_lim
dataid,,,,,
171,-0.005,0.878267,0.878267,3.4,10.84
171,-0.005,0.607600,0.607600,3.4,10.84
171,-0.005,0.717600,0.717600,3.4,10.84
171,-0.005,0.642333,0.642333,3.4,10.84
171,-0.005,0.683000,0.683000,3.4,10.84


In [90]:
df_year.to_csv('04-final-trace-2013.csv')